Leave only add events in cluster trace.

In [3]:
import pandas as pd
colnames=['timestamp', 'machine_id', 'event_type', 'event_detail', 'number_of_cpus', 'normalized_memory', 'normalized_disk_space'] 
machine_events = pd.read_csv('server_event.csv', names=colnames, header=None)
machine_events = machine_events[machine_events['event_type'] == "add"]
machine_events.to_csv('server_event_add_only.csv', index=False, header=False)
machine_events.describe()

/tmp/ipykernel_2881/2152793058.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,timestamp,machine_id,number_of_cpus,normalized_memory,normalized_disk_space
count,1313.0,1313.000000,1313.0,1313.000000,1313.000000
mean,0.0,657.000000,64.0,0.692729,0.999286
std,0.0,379.174762,0.0,0.029690,0.024972
min,0.0,1.000000,64.0,0.574788,0.095725
25%,0.0,329.000000,64.0,0.689970,1.000000
50%,0.0,657.000000,64.0,0.689970,1.000000
75%,0.0,985.000000,64.0,0.690001,1.000000
max,0.0,1313.000000,64.0,1.000000,1.000000


Find all number of instances as sum of instances from all batch tasks:

In [20]:
colnames=['task_create_time', 'task_end_time', 'job_id', 'task_id', 'number_of_instances', 'status', 'number_of_cpus_requested_per_instance_in_the_task', 'normalized_memory_requested_per_instance_in_the_task'] 
batch_tasks = pd.read_csv('batch_task.csv', names=colnames, header=None)
colnames=['start_timestamp', 'end_timestamp', 'job_id', 'task_id', 'machine_id', 'status', 'sequence_number', 'total_sequence_number', 'maximum_real_cpu_number', 'average_real_cpu_number', 'maximum_normalized_memory_usage', 'average_normalized_memory_usage'] 
batch_instances = pd.read_csv('batch_instance.csv', names=colnames, header=None)
batch_instances["end_timestamp"] = pd.to_numeric(batch_instances["end_timestamp"])
batch_instances["start_timestamp"] = pd.to_numeric(batch_instances["start_timestamp"])

print("all number of instances from tasks", sum(batch_tasks["number_of_instances"]))
print("all number of instances from instance trace", len(batch_instances))

all number of instances from tasks 11909321
all number of instances from instance trace 16094656


...

In [26]:
a = batch_instances[
    (batch_instances["start_timestamp"].notnull()) &
    (batch_instances["end_timestamp"].notnull()) &
    (batch_instances["task_id"].notnull()) &
    (batch_instances["end_timestamp"] >= batch_instances["start_timestamp"]) &
    (batch_instances["end_timestamp"] >= 0) &
    (batch_instances["start_timestamp"] >= 0)
]
len(a)

11877251